# 載入所需套件

In [1]:
import requests
import requests.packages.urllib3
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

requests.packages.urllib3.disable_warnings()

# 定義轉換日期格式函數

In [2]:
def process_date(date):
    date_list=[str(int(e)+1911) if i==0 else e for i,e in enumerate(date.split('/'))]
    return ''.join(date_list)

# 定義爬取特定股票指定月份歷史重大訊息函數

In [3]:
def news_summary_crawler(year,month,co_id):
    #year:民國年
    month='{:02d}'.format(month)
    url='https://mops.twse.com.tw/mops/web/ajax_t05st01?encodeURIComponent=1&firstin=true&TYPEK=sii&year={}&month={}&co_id={}'.format(year,month,co_id)
    header={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko'}
    response=requests.get(url,headers=header,verify=False)
    time.sleep(1.5)
    try:
        df=pd.read_html(response.text)[1]
        df.drop(df.columns[-1],axis=1,inplace=True)
        df['spoke_date']=df['發言日期'].apply(lambda x:process_date(x))
        df['spoke_time']=df['發言時間'].apply(lambda x:x.replace(':',''))

        seq_no_list=[]
        for e in df.groupby(['發言日期']).count().values[:,0]:
            for i in range(1,e+1):
                seq_no_list.append(str(i))

        df['seq_no']=seq_no_list
        df['link']=url+'&spoke_date='+df['spoke_date']+'&spoke_time='+df['spoke_time']+'&seq_no='+df['seq_no']+'&step=2&off=1'
    
    except:
        df=pd.DataFrame(columns=['公司代號','公司名稱','發言日期','發言時間','主旨','spoke_date','spoke_time','seq_no','link'])

    return df

# 定義爬取特定歷史重大訊息函數

In [4]:
def news_info_crawler(total_df):
    header={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko'}
    info_dfs=[]
    links=total_df['link'].values
    columns=['序號','發言日期','發言時間','發言人','發言人職稱','發言人電話','主旨','符合條款','事實發生日','說明']
    for link in tqdm(links):
        r=requests.get(link,headers=header,verify=False)
        soup=BeautifulSoup(r.text,'html.parser')
        datas=[(''.join(e.text.splitlines())).replace('\xa0','').replace('\u3000','').replace('\t','') for e in soup.select('.odd')]
        
        if datas==[]:
            info_df=pd.DataFrame(columns=columns)
        else:
            info_df=pd.DataFrame([datas],columns=columns)
        
        info_dfs.append(info_df)
        time.sleep(1.5)
        
    total_info_df=pd.concat(info_dfs,ignore_index=True,sort=False)
    
    return total_info_df

# 爬取特定股票指定時間範圍歷史重大訊息

In [5]:
co_id=2330
dfs=[]
for year in tqdm(range(107,109)):
    for month in range(1,13):
        df=news_summary_crawler(year,month,co_id)
        dfs.append(df)
total_df=pd.concat(dfs,ignore_index=True,sort=False)
total_df

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:53<00:00, 26.60s/it]


,公司代號,公司名稱,發言日期,發言時間,主旨,spoke_date,spoke_time,seq_no,link
0,2330,台積電,107/01/02,18:12:50,本公司代子公司台積電(中國)公告取得人民幣保本型結構性 存款商品,20180102,181250,1,https://mops.twse.com.tw/mops/web/ajax_t05st01...
1,2330,台積電,107/01/03,17:33:40,本公司代子公司台積電（南京）有限公司公告訂購機器設備,20180103,173340,1,https://mops.twse.com.tw/mops/web/ajax_t05st01...
2,2330,台積電,107/01/04,13:41:55,本公司訂購機器設備公告,20180104,134155,1,https://mops.twse.com.tw/mops/web/ajax_t05st01...
3,2330,台積電,107/01/04,13:42:14,本公司訂購工程設施公告,20180104,134214,2,https://mops.twse.com.tw/mops/web/ajax_t05st01...
4,2330,台積電,107/01/05,17:16:04,本公司代子公司台積電(南京)公告取得人民幣保本型結構性 存款商品,20180105,171604,1,https://mops.twse.com.tw/mops/web/ajax_t05st01...
5,2330,台積電,107/01/08,13:36:28,本公司訂購工程設施公告,20180108,133628,1,https://mops.twse.com.tw/mops/web/ajax_t05st01...
6,2330,台積電,107/01/08,14:32:53,公告本公司將於107年1月18日召開法人說明會,20180108,143253,2,https://mops.twse.com.tw/mops/web/ajax_t05st01...
7,2330,台積電,107/01/09,17:27:17,本公司訂購廠務設備公告,20180109,172717,1,https://mops.twse.com.tw/mops/web/ajax_t05st01...
8,2330,台積電,107/01/10,14:47:28,台積公司2017年12月營收報告,20180110,144728,1,https://mops.twse.com.tw/mops/web/ajax_t05st01...
9,2330,台積電,107/01/11,13:39:38,本公司訂購機器設備公告,20180111,133938,1,https://mops.twse.com.tw/mops/web/ajax_t05st01...


# 爬取特定歷史重大消息

In [6]:
total_info_df=news_info_crawler(total_df.iloc[:50,])
total_info_df

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:37<00:00,  2.05s/it]


,序號,發言日期,發言時間,發言人,發言人職稱,發言人電話,主旨,符合條款,事實發生日,說明
0,1,107/01/02,18:12:50,何麗梅,資深副總經理暨財務長,03-563-6688,本公司代子公司台積電(中國)公告取得人民幣保本型結構性存款商品,第20款,107/01/02,1.標的物之名稱及性質（屬特別股者，並應標明特別股約定發行條件，如股息率等）:美元利率掛鉤人...
1,1,107/01/03,17:33:40,何麗梅,資深副總經理暨財務長,03-563-6688,本公司代子公司台積電（南京）有限公司公告訂購機器設備,第20款,107/01/03,1.標的物之名稱及性質（如坐落台中市北區ＸＸ段ＸＸ小段土地）:機器設備。2.事實發生日:10...
2,1,107/01/04,13:41:55,何麗梅,資深副總經理暨財務長,03-563-6688,本公司訂購機器設備公告,第20款,107/01/04,1.標的物之名稱及性質（如坐落台中市北區ＸＸ段ＸＸ小段土地）:機器設備。2.事實發生日:10...
3,2,107/01/04,13:42:14,何麗梅,資深副總經理暨財務長,03-563-6688,本公司訂購工程設施公告,第20款,107/01/04,1.標的物之名稱及性質（如坐落台中市北區ＸＸ段ＸＸ小段土地）:工程設施。2.事實發生日:10...
4,1,107/01/05,17:16:04,何麗梅,資深副總經理暨財務長,03-563-6688,本公司代子公司台積電(南京)公告取得人民幣保本型結構性存款商品,第20款,107/01/05,1.標的物之名稱及性質（屬特別股者，並應標明特別股約定發行條件，如股息率等）:可展期存款渣打...
5,1,107/01/08,13:36:28,何麗梅,資深副總經理暨財務長,03-563-6688,本公司訂購工程設施公告,第20款,107/01/08,1.標的物之名稱及性質（如坐落台中市北區ＸＸ段ＸＸ小段土地）:工程設施。2.事實發生日:10...
6,2,107/01/08,14:32:53,何麗梅,資深副總經理暨財務長,03-563-6688,公告本公司將於107年1月18日召開法人說明會,第12款,107/01/18,符合條款第四條第XX款：12事實發生日：107/01/181.召開法人說明會之日期：107/...
7,1,107/01/09,17:27:17,何麗梅,資深副總經理暨財務長,03-563-6688,本公司訂購廠務設備公告,第20款,107/01/09,1.標的物之名稱及性質（如坐落台中市北區ＸＸ段ＸＸ小段土地）:廠務設備。2.事實發生日:10...
8,1,107/01/10,14:47:28,何麗梅,資深副總經理暨財務長,03-563-6688,台積公司2017年12月營收報告,第49款,107/01/10,1.事實發生日:107/01/102.公司名稱:台灣積體電路製造股份有限公司。3.與公司關係...
9,1,107/01/11,13:39:38,何麗梅,資深副總經理暨財務長,03-563-6688,本公司訂購機器設備公告,第20款,107/01/11,1.標的物之名稱及性質（如坐落台中市北區ＸＸ段ＸＸ小段土地）:機器設備。2.事實發生日:10...
